In [23]:
import sys 
import os 
import re 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from pathlib import Path

sys.path.append("/Users/ngokykhanhthu/Documents/Mental_Health_n_LLMs")
from mentalLLM.config import RAW_DIR, INTERIM_DIR, PROCESSED_DIR, FINAL_DIR, DATASET, DATA_KEYWORD, CHAT_GPT_QUESTION
from chat_gpt_scraper import remove_white_spaces, is_unix_time, convert_unix_time, find_links, scrape_link, retrieve_the_json_portion, extract_json_object, create_csv_files, extract_msg

In [24]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [25]:
directory_path = Path("../data/0.raw")

file_path = directory_path / "data_6.csv"
data = pd.read_csv(file_path)

In [26]:
# rename col 
data.rename(columns=lambda x: re.sub('LLM_S_7_2','llm_other_file',x), inplace=True)

# extracting valid responses 

a valid response is one that gives us at least 1 chat 

In [27]:
# remove the meta data rows 
data = data.drop([0,1])
data.head(3)
# remove meta data cols
data = data.drop(['IPAddress', 'Duration (in seconds)', 'UserLanguage'], axis = 1)
data = data.drop(data.loc[:, "ResponseId": "LocationLongitude"].columns, axis= 1)

# remove stuff that i put in
data = data[data["DistributionChannel"] != "preview"]
data = data[data["1_llm_gpt"] != "https://chatgpt.com/share/68de6922-d988-8001-8f7c-ab05d2f633d7"]

# drop empty responses
data = data.dropna(subset=data.loc[:, "eligibility_2":].columns, how='all')


data = data.dropna(subset=data.loc[:, "1_llm_gpt":"5_llm_project_file_Type"].columns, how='all')
data.shape[0]

data.to_csv("../data/4.analysis/1.0-valid_data.csv")

In [28]:
data.head(2)

,StartDate,EndDate,Status,Progress,Finished,RecordedDate,DistributionChannel,Q_RecaptchaScore,consent_1,eligibility_1,eligibility_1.1,eligibility_2,general_num_chat,1_llm_type,1_llm_gpt,1_llm_claude,1_llm_cursor_Id,1_llm_cursor_Name,1_llm_cursor_Size,1_llm_cursor_Type,1_llm_copilot_Id,1_llm_copilot_Name,1_llm_copilot_Size,1_llm_copilot_Type,1_llm_deepseek_Id,1_llm_deepseek_Name,1_llm_deepseek_Size,1_llm_deepseek_Type,1_llm_other_name,1_llm_other_link,1_llm_other_file_Id,1_llm_other_file_Name,1_llm_other_file_Size,1_llm_other_file_Type,1_llm_project,1_llm_project_link,1_llm_project_file_Id,1_llm_project_file_Name,1_llm_project_file_Size,1_llm_project_file_Type,2_llm_type,2_llm_gpt,2_llm_claude,2_llm_cursor_Id,2_llm_cursor_Name,2_llm_cursor_Size,2_llm_cursor_Type,2_llm_copilot_Id,2_llm_copilot_Name,2_llm_copilot_Size,2_llm_copilot_Type,2_llm_deepseek_Id,2_llm_deepseek_Name,2_llm_deepseek_Size,2_llm_deepseek_Type,2_llm_other_name,2_llm_other_link,2_llm_other_file_Id,2_llm_other_file_Name,2_llm_other_file_Size,2_llm_other_file_Type,2_llm_project,2_llm_project_link,2_llm_project_file_Id,2_llm_project_file_Name,2_llm_project_file_Size,2_llm_project_file_Type,3_llm_type,3_llm_gpt,3_llm_claude,3_llm_cursor_Id,3_llm_cursor_Name,3_llm_cursor_Size,3_llm_cursor_Type,3_llm_copilot_Id,3_llm_copilot_Name,3_llm_copilot_Size,3_llm_copilot_Type,3_llm_deepseek_Id,3_llm_deepseek_Name,3_llm_deepseek_Size,3_llm_deepseek_Type,3_llm_other_name,3_llm_other_link,3_llm_other_file_Id,3_llm_other_file_Name,3_llm_other_file_Size,3_llm_other_file_Type,3_llm_project,3_llm_project_link,3_llm_project_file_Id,3_llm_project_file_Name,3_llm_project_file_Size,3_llm_project_file_Type,4_llm_type,4_llm_gpt,4_llm_claude,4_llm_cursor_Id,4_llm_cursor_Name,4_llm_cursor_Size,4_llm_cursor_Type,4_llm_copilot_Id,4_llm_copilot_Name,4_llm_copilot_Size,4_llm_copilot_Type,4_llm_deepseek_Id,4_llm_deepseek_Name,4_llm_deepseek_Size,4_llm_deepseek_Type,4_llm_other_name,4_llm_other_link,4_llm_other_file_Id,4_llm_other_file_Name,4_llm_other_file_Size,4_llm_other_file_Type,4_llm_project,4_llm_project_link,4_llm_project_file_Id,4_llm_project_file_Name,4_llm_project_file_Size,4_llm_project_file_Type,5_llm_type,5_llm_gpt,5_llm_claude,5_llm_cursor_Id,5_llm_cursor_Name,5_llm_cursor_Size,5_llm_cursor_Type,5_llm_copilot_Id,5_llm_copilot_Name,5_llm_copilot_Size,5_llm_copilot_Type,5_llm_deepseek_Id,5_llm_deepseek_Name,5_llm_deepseek_Size,5_llm_deepseek_Type,5_llm_other_name,5_llm_other_link,5_llm_other_file_Id,5_llm_other_file_Name,5_llm_other_file_Size,5_llm_other_file_Type,5_llm_project,5_llm_project_link,5_llm_project_file_Id,5_llm_project_file_Name,5_llm_project_file_Size,5_llm_project_file_Type,chat_purpose,chat_llmcode_used_1,chat_feelings_1,chat_feelings_2,chat_feelings_3,chat_feelings_4,chat_feelings_5,chat_feelings_6,chat_feelings_7,chat_feelings_8,chat_feelings_text,efficacy_1_1,efficacy_1_2,efficacy_1_3,efficacy_1_4,efficacy_1_5,efficacy_1_6,efficacy_2_1,efficacy_2_2,efficacy_2_3,efficacy_2_4,efficacy_2_5,efficacy_3_1,efficacy_3_2,efficacy_3_3,efficacy_3_4,efficacy_3_5,efficacy_3_6,efficacy_4_1,efficacy_4_2,efficacy_4_3,prgming_years_1,prgming_occupation,prgming_freq_code,prgming_freq_llm,prgming_degree,prgming_major,prgming_dsa,prgming_swe,prgming_swe_3_TEXT,prgming_job_title,prgming_job_code_hz,prgming_experience,prgming_tool_comfort_1,prgming_tool_comfort_2,prgming_tool_comfort_3,prgming_tool_comfort_4,prgming_tool_comfort_5,prgming_tool_comfort_6,pmh_name,pmh_name_7_TEXT,pmh_validated_1,pmh_validated_2,pmh_validated_3,pmh_validated_4,pmh_validated_5,pmh_validated_6,pmh_validated_7,pmh_validated_8,pmh_validated_9,neurotype_name,neurotype_name_5_TEXT,neurotype_validated_1,neurotype_validated_2,neurotype_validated_3,neurotype_validated_4,neurotype_validated_5,neurotype_validated_6,dem_gender,dem_gender_3_TEXT,dem_institution,dem_institution_7_TEXT,Logistics_1,Logistics_2
7,2025-10-09 15:19:12,2025-10-09 15:25:07,0,100,1,2025-10-09 15:25:08,anonymous,1,1,23,2,2,1,1,https://chatgpt.co

# check for nas 

In [29]:
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [30]:
print("NAs per column:")
print(data.loc[:, "general_num_chat":].isna().sum())

NAs per column:
general_num_chat            0
1_llm_type                  0
1_llm_gpt                   6
1_llm_claude               19
1_llm_cursor_Id            20
1_llm_cursor_Name          20
1_llm_cursor_Size          20
1_llm_cursor_Type          20
1_llm_copilot_Id           19
1_llm_copilot_Name         19
1_llm_copilot_Size         19
1_llm_copilot_Type         19
1_llm_deepseek_Id          20
1_llm_deepseek_Name        20
1_llm_deepseek_Size        20
1_llm_deepseek_Type        20
1_llm_other_name           17
1_llm_other_link           17
1_llm_other_file_Id        20
1_llm_other_file_Name      20
1_llm_other_file_Size      20
1_llm_other_file_Type      20
1_llm_project               0
1_llm_project_link         18
1_llm_project_file_Id      18
1_llm_project_file_Name    18
1_llm_project_file_Size    18
1_llm_project_file_Type    18
2_llm_type                 18
2_llm_gpt                  18
2_llm_claude               20
2_llm_cursor_Id            20
2_llm_cursor_Name       

In [31]:
pd.reset_option('display.max_rows')
pd.reset_option('display.width')

# check if there is any bot response

In [32]:
# data["Q_RecaptchaScore"] = data["Q_RecaptchaScore"].astype('category')
# col.value_counts()

# data[data["Q_RecaptchaScore"] != 1]
# # great none are actual bots 

In [33]:
# gender
gender_map = {"1": "Man", 
            "2": "Woman", 
            "3": "Other", 
            "4": "Prefer not to say"}

data.replace({"dem_gender": gender_map}, inplace=True)
data["dem_gender"].value_counts()

dem_gender
Woman    13
Man       2
Other     1
Name: count, dtype: int64

In [34]:
total_gender = data["dem_gender"].count().item()

data["dem_gender"].value_counts() *100/ total_gender

dem_gender
Woman    81.25
Man      12.50
Other     6.25
Name: count, dtype: float64

In [35]:
data[data["dem_gender"] == "Other"]["dem_gender_3_TEXT"].value_counts()

Series([], Name: count, dtype: int64)

In [36]:
# institution 
inst_map = {"1": "Mount Holyoke College", 
            "2": "Smith College", 
            "3": "UMass Amherst", 
            "4": "Hampshire College", 
            "5": "Wellesley College", 
            "6": "University of Michigan", 
            "7": "Other"}

data.replace({"dem_institution": inst_map}, inplace=True)


data["dem_institution"].value_counts()

dem_institution
Mount Holyoke College    7
UMass Amherst            6
Smith College            2
Other                    1
Name: count, dtype: int64

In [37]:
data[data["dem_institution"] == "Other"]["dem_institution_7_TEXT"].value_counts()

dem_institution_7_TEXT
Swinburne University (Australia)    1
Name: count, dtype: int64

In [38]:
total_inst = data["dem_institution"].count().item()

data["dem_institution"].value_counts() *100/ total_inst

dem_institution
Mount Holyoke College    43.75
UMass Amherst            37.50
Smith College            12.50
Other                     6.25
Name: count, dtype: float64

In [39]:
# age
data["eligibility_1"] = data["eligibility_1"].astype("int64")

data[["eligibility_1"]].mean()
data[["eligibility_1"]].max()

# data[["eligibility_1"]].median()

eligibility_1    32
dtype: int64

# programming experience 


In [40]:
data["prgming_major"].value_counts()

Series([], Name: count, dtype: int64)

In [41]:
# programming years  
# data = data.dropna(subset=data.loc[:, "eligibility_2":].columns, how='all')

years = data.dropna(subset = ["prgming_years_1"])
years["prgming_years_1"] = years["prgming_years_1"].astype("float")

# years["prgming_years_1"].value_counts

print("mean: ", years[["prgming_years_1"]].mean())
print("max: ", years[["prgming_years_1"]].max())
print("min: ", years[["prgming_years_1"]].min())
print("median: ", years[["prgming_years_1"]].median())

mean:  prgming_years_1    5.125
dtype: float64
max:  prgming_years_1    12.0
dtype: float64
min:  prgming_years_1    0.0
dtype: float64
median:  prgming_years_1    4.0
dtype: float64


/var/folders/mb/nmbbbnyj4sdd7vnkcbt46n8h0000gn/T/ipykernel_7542/3752298903.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  years["prgming_years_1"] = years["prgming_years_1"].astype("float")


In [42]:
def map_occupations(value, custom_map):
    if pd.isna(value):
        return value
    codes = str(value).split(',')
    mapped = [custom_map.get(code.strip(), code.strip()) for code in codes]
    return ', '.join(mapped)


In [43]:
occupation_map = {"1": "Self-employed", 
            "2": "Undergraduate student", 
            "3": "Graduate student (Masters, PhD)", 
            "4": "Working professional", 
            "5": "Other"}

# data.replace({"prgming_occupation": occupation_map}, inplace=True)

data['prgming_occupation'] = data['prgming_occupation'].apply(
    lambda x: map_occupations(x, occupation_map)
)

answers = ['Self-employed', 'Undergraduate student', 
            'Graduate student (Masters, PhD)', 'Working professional', 'Other']


result = pd.DataFrame({
    'Type of occupation': answers,
    'count': [data['prgming_occupation'].str.contains(re.escape(answer), na=False).sum() for answer in answers]
})

result

,Type of occupation,count
0,Self-employed,1
1,Undergraduate student,11
2,"Graduate student (Masters, PhD)",4
3,Working professional,1
4,Other,0


In [44]:
data["prgming_occupation"].count().item()


16

In [45]:
1/16

0.0625

In [46]:
# data[["prgming_freq_code"]].dropna().mean()

col = data[["prgming_freq_code"]].value_counts()

# col = data[["prgming_freq_code"]].dropna()
# col.mean()

In [47]:
data["prgming_freq_llm"].value_counts()

prgming_freq_llm
4    6
3    4
1    3
2    2
5    1
Name: count, dtype: int64

In [48]:
data["prgming_degree"].value_counts()

prgming_degree
2    10
4     6
Name: count, dtype: int64

In [49]:
data["prgming_dsa"].value_counts()

prgming_dsa
3    14
1     2
Name: count, dtype: int64

In [50]:
14/16

0.875

In [51]:
data["prgming_swe"].value_counts()

prgming_swe
1    2
Name: count, dtype: int64

In [52]:
data["prgming_job_title"].value_counts()

prgming_job_title
Intern    1
Name: count, dtype: int64

In [53]:
data["prgming_job_code_hz"].value_counts()

prgming_job_code_hz
3    1
5    1
Name: count, dtype: int64

In [54]:
data["prgming_experience"].value_counts()

prgming_experience
1    1
2    1
Name: count, dtype: int64

In [55]:
# TODO efficacy 

# pmh 


In [56]:
data['neurotype_name'].count().item()

8

In [57]:
1/8

0.125

In [58]:
# TODO: neurotype_validated

In [59]:
# TODO: pmh_name 
answers = ['Attention-Deficit/Hyperactivity Disorder (ADHD)', 'Autism Spectrum Disorder (ASD)', 
            'Dyslexia', 'Obsessive-compulsive disorder (OCD)', 'Other']

result = pd.DataFrame({
    'neurodivergent conditions': answers,
    'count': [data['neurotype_name'].str.contains(re.escape(answer), na=False).sum() for answer in answers]
})
result


,neurodivergent conditions,count
0,Attention-Deficit/Hyperactivity Disorder (ADHD),0
1,Autism Spectrum Disorder (ASD),0
2,Dyslexia,0
3,Obsessive-compulsive disorder (OCD),0
4,Other,0


In [60]:
# TODO: pmh_validated

# neurotype 

In [61]:
neuro_map = {"1": 'Attention-Deficit/Hyperactivity Disorder (ADHD)', 
                "2": 'Autism Spectrum Disorder (ASD)', 
                "3": 'Dyslexia', 
                "4": 'Obsessive-compulsive disorder (OCD)', 
                "5": 'Other'}

# data.replace({"prgming_occupation": occupation_map}, inplace=True)

data['neurotype_name'] = data['neurotype_name'].apply(
    lambda x: map_occupations(x, neuro_map)
)

answers = ['Attention-Deficit/Hyperactivity Disorder (ADHD)', 'Autism Spectrum Disorder (ASD)', 
            'Dyslexia', 'Obsessive-compulsive disorder (OCD)', 'Other']

result = pd.DataFrame({
    'Type of occupation': answers,
    'count': [data['neurotype_name'].str.contains(re.escape(answer), na=False).sum() for answer in answers]
})

result

,Type of occupation,count
0,Attention-Deficit/Hyperactivity Disorder (ADHD),4
1,Autism Spectrum Disorder (ASD),2
2,Dyslexia,0
3,Obsessive-compulsive disorder (OCD),4
4,Other,1


In [64]:
occupation_map = {"1": "Self-employed", 
            "2": "Undergraduate student", 
            "3": "Graduate student (Masters, PhD)", 
            "4": "Working professional", 
            "5": "Other"}

# data.replace({"prgming_occupation": occupation_map}, inplace=True)

data['prgming_occupation'] = data['prgming_occupation'].apply(
    lambda x: map_occupations(x, occupation_map))

answers = ['Self-employed', 'Undergraduate student', 
            'Graduate student (Masters, PhD)', 'Working professional', 'Other']


result = pd.DataFrame({
    'Type of occupation': answers,
    'count': [data['prgming_occupation'].str.contains(re.escape(answer), na=False).sum() for answer in answers]
})

result



,Type of occupation,count
0,Self-employed,1
1,Undergraduate student,11
2,"Graduate student (Masters, PhD)",4
3,Working professional,1
4,Other,0


# RQ1 

In [ ]:
# how many unique links weve got
# list them 
# check if any of them is urs 
# how many are gpt links? 

list_of_cols = data.columns.tolist()

def extract_links(model_name):
    list_of_col_names = []
    pattern = re.compile(f".*llm_{model_name}$")
    for x in list_of_cols: 
        if pattern.match(x): 
            list_of_col_names.append(x)

    ret
selected_cols = data[list_of_col_names]

NameError: name 'list_of_col_names' is not defined

In [ ]:
extract_links("gpt")

In [65]:
neuro_map = {"1": 'Anxiety', 
                "2": 'Depression', 
                "3": 'Post-Traumatic Stress Disorder (PTSD)', 
                "4": 'Bipolar disorder', 
                "5": 'Borderline personality disorder (BPD)', 
                "6": "Other"}

# data.replace({"prgming_occupation": occupation_map}, inplace=True)

data['pmh_name'] = data['pmh_name'].apply(
    lambda x: map_occupations(x, neuro_map)
)

answers = ['Anxiety', 'Depression', 
            'Post-Traumatic Stress Disorder (PTSD)', 'Bipolar disorder', 'Borderline personality disorder (BPD)', "Other"]

result = pd.DataFrame({
    'Type of occupation': answers,
    'count': [data['pmh_name'].str.contains(re.escape(answer), na=False).sum() for answer in answers]
})

result

,Type of occupation,count
0,Anxiety,14
1,Depression,5
2,Post-Traumatic Stress Disorder (PTSD),0
3,Bipolar disorder,1
4,Borderline personality disorder (BPD),0
5,Other,0


In [67]:
data["pmh_name"].count().item()

15

In [70]:
1/15

0.06666666666666667